In [1]:
import argparse
import torch
import sys
import random
import os
import threadpool

sys.path.insert(0, sys.path[0] + '/..')
# sys.path.append("..")
import utils

In [2]:
class arguments():
    def __init__(self):
        self.data='/iesl/canvas/hanqingli/NSD_for_sentence_embedding/data/wiki2016_min100/'
        self.save='/iesl/canvas/hanqingli/NSD_for_sentence_embedding/data/wiki2016_min100/tensors'
        self.max_sent_len=50
        self.multi_sent=False
        self.max_target_num=30
        self.max_sent_num=1000
        self.target_num_prev = 5
        self.target_num_next=5
        self.seed=1038
        self.stop_word_file='/iesl/canvas/hanqingli/NSD_for_sentence_embedding/data/stop_word_list'
args=arguments()

In [3]:
print(args)

random.seed(args.seed)

if not os.path.exists(args.save):
    os.makedirs(args.save)

summarize = {}

In [4]:
def convert_stop_to_ind(f_in, w_d2_ind_freq):
    stop_word_set = set()
    for line in f_in:
        w = line.rstrip()
        if w in w_d2_ind_freq:
            stop_word_set.add(w_d2_ind_freq[w][0])
    return stop_word_set


In [5]:
def convert_stop_to_ind_lower(f_in, idx2word_freq):
    stop_word_org_set = set()
    for line in f_in:
        w = line.rstrip()
        stop_word_org_set.add(w)
    stop_word_set = set()
    for idx, (w, freq) in enumerate(idx2word_freq):
        if w.lower() in stop_word_org_set:
            stop_word_set.add(idx)
    return stop_word_set


In [6]:
def load_w_ind(f_in, max_sent_num, max_sent_len):
    w_ind_corpus = []
    last_sent = ''
    num_duplicated_sent = 0
    num_too_long_sent = 0

    for line in f_in:
        current_sent = line.rstrip()
        if current_sent == last_sent:
            num_duplicated_sent += 1
            continue
        last_sent = current_sent
        fields = current_sent.split(' ')
        if len(fields) > max_sent_len:
            num_too_long_sent += 1
            continue
        w_ind_corpus.append([int(x) for x in fields])
        if len(w_ind_corpus) % 1000000 == 0:
            print(len(w_ind_corpus))
            sys.stdout.flush()
        if len(w_ind_corpus) > max_sent_num:
            break
    print("Finish loading {} sentences. While removing {} duplicated and {} long sentences".format(len(w_ind_corpus),
                                                                                                   num_duplicated_sent,
                                                                                                   num_too_long_sent))
    return w_ind_corpus


In [7]:
corpus_input_name = args.data + "corpus_index"
dictionary_input_name = args.data + "dictionary_index"

In [8]:
with open(dictionary_input_name) as f_in:
    idx2word_freq = utils.load_idx2word_freq(f_in)

max_ind = len(idx2word_freq)

if max_ind >= 2147483648:
    print("Will cause overflow")
    sys.exit()

store_type = torch.int32


In [9]:
with open(args.stop_word_file) as f_in:
    # stop_ind_set = convert_stop_to_ind(f_in, w_d2_ind_freq)
    stop_ind_set = convert_stop_to_ind_lower(f_in, idx2word_freq)

with open(corpus_input_name) as f_in:
    w_ind_corpus = load_w_ind(f_in, args.max_sent_num, args.max_sent_len)


Finish loading 1001 sentences. While removing 0 duplicated and 76 long sentences


In [12]:
corpus_size = len(w_ind_corpus) - 2
# args.max_target_num+args.max_sent_len
print("Allocating {} bytes".format(corpus_size * (args.max_target_num + args.max_sent_len) * 4))
all_features = torch.zeros(corpus_size, args.max_sent_len, dtype=store_type)
all_targets = torch.zeros(corpus_size, args.max_sent_len, dtype=store_type)


Allocating 319680 bytes


In [16]:
random_selection_num = 0


# for i in range(1,len(w_ind_corpus)-1):
    
def match(i):
    output_i = i - 1
    if args.multi_sent:
        current_len = 0
        feature_list = []
        for j in range(i,len(w_ind_corpus)-1):
            w_ind_list = w_ind_corpus[j][:-1] #excluding <eos>
            sent_len = len(w_ind_list)
            current_len_prev = current_len
            current_len += sent_len
            if current_len > args.max_sent_len - 1:
                break
            feature_list += w_ind_list
        if current_len > args.max_sent_len - 1:
            current_len = current_len_prev
        feature_list.append(w_ind_corpus[j-1][-1])
        next_sent_ind = j
    else:
        feature_list = w_ind_corpus[i]
#         current_len = len(feature_list) - 1
        next_sent_ind = i + 1
#     print(len(feature_list))
    all_features[output_i,-len(feature_list):] = torch.tensor(feature_list,dtype = store_type)
    prev_w_ind_list = w_ind_corpus[i-args.target_num_prev:i-1]
    next_w_ind_list = w_ind_corpus[next_sent_ind:next_sent_ind+args.target_num_next-1]
    target_list = prev_w_ind_list+next_w_ind_list
    target_list = sum(target_list,[])

    target_w_list = []
    for w in target_list:
        if w not in stop_ind_set:
            target_w_list.append(w)
 
    # Count the num of appearance of feature words
    word_count = []
    from collections import Counter
    counter = Counter(target_w_list)
    for cnt, word in enumerate(feature_list):
        number = counter[word]
        word_count.append(number)

    all_targets[output_i, -len(word_count):] = torch.tensor(word_count, dtype = store_type)
#     for row in all_targets:
#         num = int(torch.sum(row))
#         print(num)
#         if num not in summarize:
#             summarize[num]=1
#         else:
#             summarize[num] += 1

pool=threadpool.ThreadPool(32)
requests = threadpool.makeRequests(match, range(1,len(w_ind_corpus)-1))
[pool.putRequest(req) for req in requests]
pool.wait()


    

9
4
3
7
38
0
47
1
9
8
4
9

0
5

4
0
4
4
0
3

10
1
11
5
8
04

4
9
5
0
911
7
1
19
5
4
8
1
8
4
0
3
1


4

31
0
3
1
7

3
0
30
0
7
0
6
4
3
10
6
5
118
10
7
0
7
9
5
9
7
4
7
11
9
0
0
8
8
3

1
5
5
4
260
56
1
3
3
1
4
3


4

3
9
2
8


3

4
86
6
7
4
3
1
10
3
33
4
4
3
8

0
7
4
3
4
5
3
1
1
0

1
9
7
4
3
1
9
5
8
1
0
3
5
7
6
6
2
1
0
0
0
1
0
9
0
0
1
7
8
0
11
9
7
0
0
4
2
3
9
34
81
0
6
4
10
0
9

1
9
3
04
10
7
0
4

0
8
7
3
11
2
2
0
0
7
0
5
1
12
9
7
112
611
6

1
7
7
1
6
3
7

5
0
3
5
5
3
4
6
4
5
8
6
0
0
1
57
0
0
6
3
1
44
3
0
4
6

4
1
4
10
84
2
4
2
7
9
2
7
1
7
0
3
110
3
0
6
5
8
0
3
10

710
0
62
7

3
5
1

1
0
4
11
1
3
5
4
5
20
0
411

9
0
4
7


4
0
34
6
8
0
34
2
55
8
11
0
0
4
7
3
10
6
5
4
13
1

3
9
3
0
4
3
3
5
1
0
8

9

0
4
0

2
3
5
1
913
4
9
9
3

0
5
6
6
107
4
5
3

7
4

5
1
0

0
1
3
0
1
0
06
1
6
33
1
7
90
0
4
9
3
1
1
7
4

0
6
9

5
10
0
0
0
3
3
0
0
7
6
11
9
1
3
93
3
04
1
4
0
6
3
3
0
0

3
4
10
4
1

60
5
4
4
1
1

11
6
1
11
3

1
6
3
5
11
9
11
23

2
1
8
10
8
0


7
1
7
6
0
11
5
1
4
13
8
0
2
3
3
4
2
3
67
11


0
1
812

100
1
2
0
2
5
4
2
4
4
3
2
0
0
2
11
2
22
2
5
140
231
3
103


0
84
411
0
4
4
2
4
48
5
0
10
1
0

208
23

2
20

1
44
3
5
0
3
6
2
5

23

4
00

4
10
7
5

2

0
3
2
25
4
05
116
0
20

1
4
7
1
2
2
16
0
7
4
5
8
35
6
3
5
0
10
5
6
16
16
4
2
38
11
4
5
8
8
8
0

7
13
4
6
4
13
4
0
0
1
7

3
0
10
16
8
13
0
1
2
2
40
20
0
23
2
2
53
16

7
7
1
23
3
10
3
2
1
7
20
2
2

1
2
23
165
04
1
11
16
4
7
16


03
0
4
11
4
5
11
1
2620

3
4
3
13
18
0

4
5
7
23
0
1
8
3
8


4
216
3
5

4
4
0
11
3
211
5
20
16
07
7
5
4
4
0
3
8
2
10

1310
10
8
4237

2
0
0
0
55
2

0
23
3
11

8
5
3
13
16
0
5
0
5
23
20
4

8
7

7
0
0
0
5
163
20
0
5
8
231
2
16
81
5
4
1
3
12
2

5
0


8
7
22
8
2


16
2
10
4
10
10
20
16
2
7

16
8
2
11
8
25
3
7
3
5
102
2
0
4
2
1
6
13
16
3
4
3
4
4
13
6
011
8
8
7

0
8
10
00
16
10
8
0
70
105

5
38
13
16
8
4

13
2
10
3
1
26515

5
1
8
2
13
2
1
20
8
5

10
8
7

5
7
0

7
8
3
2
8
8

2

2
16
10
4
223
0
2
3
10
11
8
6
11
0
4
0
0
0
7416
10
10
5
5
3
2
23
20
16
13
3
2
5
0
3
0

7

11
80
13
2
016
5
1

3
8
10

20

8
4
10
4
0
10
0
5
8
8
7


In [ ]:
# del w_ind_corpus
training_output_name = os.path.join(args.save, "train.pt")
val_org_output_name = os.path.join(args.save, "val_org.pt")
test_org_output_name = os.path.join(args.save, "test_org.pt")
val_shuffled_output_name = os.path.join(args.save, "val_shuffled.pt")
test_shuffled_output_name = os.path.join(args.save, "test_shuffled.pt")

testing_size_ratio = 0.05
testing_size = int(corpus_size * testing_size_ratio)
print("Testing size: {}".format(testing_size))


In [ ]:
def store_tensors(f_out, tensor1, tensor2):
    torch.save([tensor1, tensor2], f_out)


In [ ]:
# with open(test_org_output_name, 'wb') as f_out:
#     store_tensors(f_out, all_features[-testing_size:, :].clone(), all_targets[-testing_size:, :].clone())

# with open(val_org_output_name, 'wb') as f_out:
#     store_tensors(f_out, all_features[-2 * testing_size:-testing_size, :].clone(),
#                   all_targets[-2 * testing_size:-testing_size, :].clone())

# rest_size = corpus_size - 2 * testing_size
# shuffle_ind = list(range(rest_size))
# random.shuffle(shuffle_ind)

# with open(test_shuffled_output_name, 'wb') as f_out:
#     store_ind = shuffle_ind[-testing_size:]
#     store_tensors(f_out, all_features[store_ind, :], all_targets[store_ind, :])

# with open(val_shuffled_output_name, 'wb') as f_out:
#     store_ind = shuffle_ind[-2 * testing_size:-testing_size]
#     store_tensors(f_out, all_features[store_ind, :], all_targets[store_ind, :])

# with open(training_output_name, 'wb') as f_out:
#     store_ind = shuffle_ind[:rest_size - 2 * testing_size]
#     print("Training size: {}".format(len(store_ind)))
#     store_tensors(f_out, all_features[store_ind, :], all_targets[store_ind, :])


In [ ]:
print(summarize)